# Addaction Analysis

Addaction have contacted us off the back of the blog on the impact on NCSO on local authorties and charities. We estimated in our blog that the impact of NCSO on Addaction is an additional £226,144.28 on top of a "usual" tariff cost of £47 257.92 giving a total of £273,402.2 in 2018. Addaction have estimated the impact on them is £403,474.59.

In [14]:
import pandas as pd
import numpy as np
from ebmdatalab import bq

In [16]:
## SQL query to extract additonal costs of NHS price concessions. 
## This is a modfied version of the SQL query from OpenPrescribing.net NHS price concession calculator site i.e. done at a "organisational level" 
sql = '''SELECT
  ncso.date AS month,
  product.bnf_code AS bnf_code,
  product.name AS product_name,
  rx.quantity AS quantity,
  rx.pct AS pct,
  ccg.org_type,
  ccg.name,
  dt.price_pence
    * rx.quantity
    * CASE WHEN
        -- For some presentations "quantity" means "number of packs" rather
        -- than e.g. tablets. In these cases we don't want to divide by the
        -- quantity value of a pack. This is implemented via a flag in our
        -- databse but this data isn't in BiqQuery so we just have a hardcoded
        -- list of BNF codes here
        product.bnf_code in ('0206010F0AACJCJ')
      THEN
        1
      ELSE
        1 / vmpp.qtyval
      END
    -- This is the "discount factor" which applies the National Average 7.2%
    -- discount to estimate Actual Cost from Net Ingredient Cost and also
    -- converts figures from pence to pounds
    * 0.00928
    AS tariff_cost,
  COALESCE(ncso.price_concession_pence - dt.price_pence, 0)
    * rx.quantity
    * CASE WHEN
        product.bnf_code in ('0206010F0AACJCJ')
      THEN
        1
      ELSE
        1 / vmpp.qtyval
      END
    * 0.00928
    AS additional_cost,
  ncso.date != rx.month AS is_projection
FROM
  dmd.ncsoconcession AS ncso
JOIN
  dmd.tariffprice AS dt
ON
  ncso.vmpp = dt.vmpp AND ncso.date = dt.date
JOIN
  dmd.product AS product
ON
  dt.product=product.dmdid
JOIN
  dmd.vmpp AS vmpp
ON
  vmpp.vppid=ncso.vmpp
JOIN
  hscic.presentation AS presentation
ON
  presentation.bnf_code = product.bnf_code
JOIN
  hscic.prescribing AS rx
ON
  rx.bnf_code = product.bnf_code
JOIN
  hscic.ccgs AS ccg #this joins to our CCG organisational data
ON
  rx.pct = ccg.code
AND
-- Where we have prescribing data for the corresponding month we use
-- that, otherwise we use the latest month of prescribing data to
-- produce an estimate
(
  rx.month = ncso.date
  OR
  (
    -- This should be set to the latest date for which we have prescribing
    -- data
    rx.month = TIMESTAMP('2019-01-01')
    AND
    ncso.date > rx.month
  )
)'''
    
df_ncso = bq.cached_read(sql, csv_path='df_ncso_nonccg.zip', compression='zip', use_cache=True)  # add `use_cache=False` to override
df_ncso.head()

TypeError: read_gbq() got an unexpected keyword argument 'compression'

In [3]:
## import "independent providers" from NHS Digital ODS codes as this is where addaction is listed.
independent_providers = pd.read_csv(r'C:\Users\bmackenna\Documents\GitHub\jupyter-notebooks\nonCCG Concessions\ephp.csv')
independent_providers.head(5)

,pct,name,national_grouping,high_level_health_geography,address line 1,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,postcode,open_date,close_date
0,AA4,INTRAHEALTH LTD,Y54,Q74,"1ST FLOOR, WILLIAM BROWN CENTRE",MANOR WAY,NaN,PETERLEE,COUNTY DURHAM,SR8 5TW,20130401,NaN
1,AA5,COMPASS WELLBEING COMMUNITY INTEREST COMPANY,Y56,Q71,STEELS LANE HEALTH CENTRE,384-388 COMMERCIAL ROAD,NaN,LONDON,GREATER LONDON,E1 0LR,20130401,NaN
2,AA6,ASSISTED CONCEPTION UNIT LTD,Y56,Q71,LEYTONSTONE HOUSE,LEYTONSTONE,NaN,LONDON,GREATER LONDON,E11 1GA,20130401,NaN
3,AA7,LEYLAND PHYSIOTHERAPY,Y54,Q84,83 BOW LANE,NaN,NaN,LEYLAND,LANCASHIRE,PR25 4YB,20130401,NaN
4,AA8,SMART CJS,Y59,Q87,BUILDING B,KIRTLINGTON BUSINESS CENTRE,"SLADE FARM, KIRTLINGTON",KIDLINGTON,OXFORDSHIRE,OX5 3JA,20130401,NaN


In [4]:
## adding in independent provider names to our prescribing dataset result from SQL
indopro = df_ncso.merge(independent_providers[['pct','name']],  how="outer", on='pct')
indopro.head()

,month,bnf_code,product_name,quantity,pct,org_type,name_x,tariff_cost,additional_cost,is_projection,name_y
0,2018-08-01,0704020J0AAACAC,Oxybutynin 5mg tablets,210.0,10A,CCG,NHS SOUTH KENT COAST CCG,4.872000,7.830000,False,NaN
1,2018-08-01,1304000Y0AAAAAA,Mometasone 0.1% cream,480.0,10A,CCG,NHS SOUTH KENT COAST CCG,26.414592,8.775168,False,NaN
2,2018-08-01,0408010H0AAACAC,Lamotrigine 25mg tablets,678.0,10A,CCG,NHS SOUTH KENT COAST CCG,14.381349,51.907680,False,NaN
3,2018-08-01,040201030AAAAAA,Risperidone 1mg tablets,716.0,10A,CCG,NHS SOUTH KENT COAST CCG,20.265664,110.962816,False,NaN
4,2018-08-01,0206030N0AAAAAA,Nicorandil 10mg tablets,600.0,10A,CCG,NHS SOUTH KENT COAST CCG,17.075200,21.715200,False,NaN


In [5]:
## clearing blanks from the datset
df2 = indopro.dropna(subset=['name_y'])
df2

,month,bnf_code,product_name,quantity,pct,org_type,name_x,tariff_cost,additional_cost,is_projection,name_y
9616108,2018-02-01,0501011P0AAASAS,Phenoxymethylpenicillin 250mg/5ml oral solutio...,1000.0,NDL,Unknown,None,53.267200,6.310400,False,SUFFOLK GP FEDERATION
9616109,2018-03-01,0501011P0AAARAR,Phenoxymethylpenicillin 125mg/5ml oral solutio...,1800.0,NDL,Unknown,None,83.018880,47.606400,False,SUFFOLK GP FEDERATION
9616110,2016-06-01,0403010X0AAADAD,Trazodone 150mg tablets,28.0,NDL,Unknown,None,22.392640,4.055360,False,SUFFOLK GP FEDERATION
9616111,2018-01-01,0403030D0AAABAB,Citalopram 10mg tablets,168.0,NDL,Unknown,None,6.681600,1.948800,False,SUFFOLK GP FEDERATION
9616112,2018-10-01,0204000R0AAAJAJ,Propranolol 40mg tablets,280.0,NDL,Unknown,None,8.259200,16.982400,False,SUFFOLK GP FEDERATION
9616113,2018-08-01,0408010W0AAADAD,Sodium valproate 500mg gastro-resistant tablets,56.0,NDL,Unknown,None,4.640742,3.154458,False,SUFFOLK GP FEDERATION
9616114,2018-05-01,0704020N0AAABAB,Tolterodine 2mg tablets,56.0,NDL,Unknown,None,1.939520,1.215680,False,SUFFOLK GP FEDERATION
9616115,2018-11-01,0501012G0AAABAB,Flucloxacillin 500mg capsules,56.0,NDL,Unknown,None,3.340800,0.612480,False,SUFFOLK GP FEDERATION
9616116,2017-12-01,0408010AEAAAGAG,Pregabalin 300mg capsules,56.0,NDL,Unknown,None,3.285120,2.746880,False,SUFFOLK GP FEDERATION
9616117,2017-12-01,0408010G0AAABAB,Gabapentin 300mg capsules,268.0,NDL,Unknown,None,5.794803,6.267341,False,SUFFOLK GP FEDERATION


In [6]:
## filtering the dataset to only include results where Addaction or similar is in the independent provider name
addaction2 = df2[df2["name_y"].str.contains("addaction", case=False)]
addaction2.head()

,month,bnf_code,product_name,quantity,pct,org_type,name_x,tariff_cost,additional_cost,is_projection,name_y
9917793,2017-03-01,0403040X0AAAAAA,Mirtazapine 30mg tablets,91.0,NI3,Unknown,None,3.830320,1.025440,False,ADDACTION
9917794,2017-02-01,0403040X0AAAAAA,Mirtazapine 30mg tablets,98.0,NI3,Unknown,None,4.124960,1.104320,False,ADDACTION
9917795,2016-11-01,0407041M0AAAAAA,Naratriptan 2.5mg tablets,6.0,NI3,Unknown,None,1.883840,20.898560,False,ADDACTION
9917796,2018-10-01,0410030A0AAAEAE,Buprenorphine 8mg sublingual tablets sugar free,832.0,NI3,Unknown,None,236.040777,1462.570423,False,ADDACTION
9917797,2018-10-01,0410030A0AAADAD,Buprenorphine 2mg sublingual tablets sugar free,343.0,NI3,Unknown,None,43.198400,195.529600,False,ADDACTION


In [7]:
## Counting how many variations on the name Addaction are retured
addaction2["name_y"].unique()

array(['ADDACTION'], dtype=object)

There is only one variation of the name Addaction in our dataset therefore we can be reasonably confident that our impact is correct as per the NHS BSA primary care prescribing/reimbursement dataset.

In [8]:
### see impact of NCSO in 2018
addaction2018 = addaction2.loc[(addaction2["month"]>="2018-01-01") & (addaction2["month"]<="2018-12-01")]
addaction2018

,month,bnf_code,product_name,quantity,pct,org_type,name_x,tariff_cost,additional_cost,is_projection,name_y
9917796,2018-10-01,0410030A0AAAEAE,Buprenorphine 8mg sublingual tablets sugar free,832.0,NI3,Unknown,None,236.040777,1462.570423,False,ADDACTION
9917797,2018-10-01,0410030A0AAADAD,Buprenorphine 2mg sublingual tablets sugar free,343.0,NI3,Unknown,None,43.198400,195.529600,False,ADDACTION
9917798,2018-09-01,0410030A0AAADAD,Buprenorphine 2mg sublingual tablets sugar free,3615.0,NI3,Unknown,None,445.698514,2118.266057,False,ADDACTION
9917799,2018-10-01,0410030A0AAADAD,Buprenorphine 2mg sublingual tablets sugar free,396.0,NI3,Unknown,None,49.873371,225.742629,False,ADDACTION
9917800,2018-05-01,0410030A0AAAEAE,Buprenorphine 8mg sublingual tablets sugar free,910.0,NI3,Unknown,None,218.358400,1729.977600,False,ADDACTION
9917802,2018-04-01,0410030A0AAAEAE,Buprenorphine 8mg sublingual tablets sugar free,951.0,NI3,Unknown,None,228.196526,73.123749,False,ADDACTION
9917803,2018-04-01,0410030A0AAADAD,Buprenorphine 2mg sublingual tablets sugar free,8365.0,NI3,Unknown,None,1031.332800,465.763200,False,ADDACTION
9917804,2018-07-01,0410030A0AAAEAE,Buprenorphine 8mg sublingual tablets sugar free,5446.0,NI3,Unknown,None,1299.571200,7371.456640,False,ADDACTION
9917805,2018-07-01,0410030A0AAAEAE,Buprenorphine 8mg sublingual tablets sugar free,481.0,NI3,Unknown,None,114.780343,651.059611,False,ADDACTION
9917806,2018-04-01,0410030A0AAAEAE,Buprenorphine 8mg sublingual tablets sugar free,68.0,NI3,Unknown,None,16.316891,5.228617,False,ADDACTION


In [9]:
### check impact for 2018
addaction2018.sum()

quantity           278206.000000
tariff_cost         47257.922650
additional_cost    226144.279588
dtype: float64

Now we will check the spend on Buprenorphine (BNF code = 0410030A0%) to see if Addaction are counting total impact instead of diferentiating between NCSO and "normal cost". The difference here is we are looking at all Buprenorphine products in all months of 2018. The previous analysis only look at products in months when they are NCSO.

In [11]:
sql = '''SELECT
  extract(year from month) AS year,
  bnf_code AS bnfcode,
  bnf_name AS bnfname,
  SUM(items) AS total_items,
  SUM(quantity) AS total_qty,
  SUM(actual_cost) AS actual_cost
FROM
  ebmdatalab.hscic.normalised_prescribing_standard AS presc
WHERE
  bnf_code LIKE "0410030A0%"  
  and
  pct = "NI3" 
GROUP BY
  year,
  bnf_code,
  bnf_name'''
    
df_addaction = bq.cached_read(sql, csv_path='df_addactionbuprenorphine.csv', use_cache=True)  # add `use_cache=False` to override
df_addaction.head()

C:\Users\bmackenna\Anaconda3\lib\site-packages\pandas_gbq\gbq.py:556: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  "verbosity", FutureWarning, stacklevel=1)


,year,bnfcode,bnfname,total_items,total_qty,actual_cost
0,2012,0410030A0AAADAD,Buprenorphine_Tab Subling 2mg S/F,708,24602,24035.80000
1,2017,0410030A0BBABAD,Subutex_Tab Subling 2mg,843,22031,18634.76891
2,2015,0410030A0AAADAD,Buprenorphine_Tab Subling 2mg S/F,8384,205332,59216.95000
3,2010,0410030A0AAAAAA,Acamprosate Calc_Tab E/C 333mg,1,28,3.72000
4,2019,0410030A0AAADAD,Buprenorphine_Tab Subling 2mg S/F,710,18859,16819.16657


In [12]:
### grouping the total cost by year
totalcost = df_addaction.groupby(['year'])['actual_cost'].sum()
totalcost

year
2010         3.72000
2011     53178.14000
2012    330436.05000
2013    315208.23000
2014    333620.98000
2015    247939.29000
2016    215990.79000
2017    171170.42258
2018    352194.52391
2019     51633.75501
Name: actual_cost, dtype: float64

In [13]:
## graphing full years - i.e. get rid of 2010 and 2019 as incomplete
totalcostfullyearsonly = totalcost.drop([2010, 2019])
ax = totalcostfullyearsonly.plot(kind='line', title = "Total Cost £ of Buprenorphine at Addaction")
ax.set_ylim(0, 400000)


(0, 400000)

Addaction spent £352,194.52 on Buprenorphine  (BNF code = 0410030A0%) as per the prescribing file which is £51,280 short of their estimate of £403,474.59. We will now clarify/ask
-  What is the time frame? 18/19 has not finished so are they estimating or is the £403k a partial year spend?
-  What is the datset? for example is this invoiced?
-  Do they have other prescribing codes where the name is not similar to Addaction?
-  Do they pay through a "recharge" mechanism? e.g. a CCG pays and then invoices addaction

Addaction have responded to say they have done an estimate for 2018/2019 and that our calculation are in line with their estimates.